# Bias, overfitting and regularization

Our machine learning algorithm learns from the training set and can then make predictions for new inputs. However, there's a risk that the algorithm has learned about the training set so well, that it will not generalize to new inputs, and consequently performs poorly. This is a major concern in machine learning that we'll address next. 

Please watch the following videos (~20 min.):
- [Regularization: the problem of overfitting](https://www.youtube.com/watch?v=u73PU6Qwl1I&list=PLLssT5z_DsK-h9vYZkQkYNWcItqhlRJLN&index=39) (9 min.). Concepts to grasp:
    * Overfitting (high variance), underfitting (high bias). 
    * Too many features (different features from the domain or constructed features like polynomials) -> risk of overfitting!! Prevention: reduce number of features by only selecting the features that really matter. This can be done manually, or automatically. Automatic methods: correlation (heat map), dimensionality reduction (PCA), .... More info [here](https://towardsdatascience.com/feature-selection-and-dimensionality-reduction-f488d1a035de). Of course, by ignoring features, important information might be lost!
    * Too few training samples -> risk for overfitting!! Counter measures: artificially increase the number of training samples.
    * With many features, plotting the data is not possible any more to see if overfitting occurs.
    * Careful model selection to prevent overfitting.
- [Regularization: cost function](https://www.youtube.com/watch?v=KvtGD37Rm5I&list=PLLssT5z_DsK-h9vYZkQkYNWcItqhlRJLN&index=40) (10 min.). Concepts to grasp: regularization. Regularization parameter lambda is a trade-off between fitting the training set wel and prevent overfitting. On other words lambda is a trade-off between bias and high variance. By convention, theta_0 is not regularized. 

# Case description
You will implement regularized logistic regression to predict whether microchips from a fabrication plant pass quality assurance (QA). During QA, each microchip goes through various tests to ensure it is functioning correctly.

Suppose you are the product manager of the factory and you have the test results for some microchips on two different tests. From these two tests, you would like to determine whether the microchips should be accepted or rejected. To help you make the decision, you have a dataset of test results on microchips fabricated in the past, from which you can build a logistic regression model.

**Exercise**:
* read the training data. Feature 1 (column 0) is the result of test 1. Feature 2 (column 1) is the result of test 2. The third column is the label and determines whether the microchip was accepted or rejected.
* save the data in ```X``` and ```y```.
* explore the training data using ```df.head()```.
* visualize the training data. x-axis: Microchip Test1, y-axis: Microchip Test2', use a mask to give passed and rejected microchips a different colour.

The plot should look similar to:
<img src="https://github.com/sjoerdteunisse/Avans-A.I-Minor/blob/master/Workshop%202/images/dataVisualization2.png?raw=true" />

The figure shows that our dataset cannot be separated into positive and negative examples by a straight line through the plot. Therefore, a straightforward application of logistic regression will not perform well on this dataset, as logistic regression will only be able to find a linear decision boundary.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# read the training data
df = pd.read_csv('ex2data2.txt', header = None)  # "df" is short for data frame

In [ ]:
# prepare the training set
X = df.iloc[:,:-1]
y = df.iloc[:,2]
y = y[:, np.newaxis]

# explore the data
df.head()

In [ ]:
# visualize the data
mask = y == 1
passed = plt.scatter(X[mask][0].values, X[mask][1].values)
failed = plt.scatter(X[~mask][0].values, X[~mask][1].values)
plt.xlabel('Microchip Test1')
plt.ylabel('Microchip Test2')
plt.legend((passed, failed), ('Passed', 'Failed'))
plt.show()

## Feature mapping
One way to fit the data better is to create more features from each data point. Hence we will map the features into all polynomial terms of $x_1$ and $x_2$ up to the sixth power.

<img src="https://github.com/sjoerdteunisse/Avans-A.I-Minor/blob/master/Workshop%202/images/featureMapping.png?raw=true"/>

As a result of this mapping, our vector of two features (the scores on two QA tests) has been transformed into a 28-dimensional vector. A logistic regression classifier trained on this higher-dimension feature vector will have a more complex decision boundary and will appear nonlinear when drawn in our 2-dimensional plot.

While the feature mapping allows us to build a more expressive classifier, it is also more susceptible to overfitting. In the next parts of the exercise, you will implement regularized logistic regression to fit the data and also see for yourself how regularization can help combat the overfitting problem.

We'll do feature mapping for polynomials up to degree 6. This can easily be done by using [sklearn.preprocessing.PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). 

In [ ]:
# feature mapping
degree = 6

from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree)
X = poly_reg.fit_transform(X)
(m, n) = X.shape

In [ ]:
# sigmoid and hypothesis functions are copied from the previous hands-on, as they are not different for regularized logistic regression

# by convention, call the function parameter z
def sigmoid(z):
    return 1/(1 + np.exp(-z))

# hypothesis
def h(X, theta):
    return sigmoid(X @ theta)  # alternative way of writing dot product

The cost function for logistic regression (previous hands-on) has been extended by a regularization term to avoid overfitting. By convention $\Theta_0$ is not used in the regularization term:


<img src="https://github.com/sjoerdteunisse/Avans-A.I-Minor/blob/master/Workshop%202/images/logisticRegressionCostFunctionRegularized.png?raw=true"/>


This is called L2 regularization, because the regularization term squares the thetas.

**Exercise**: implement the body of the regularized cost function.

In [ ]:
# regularized cost function
def Jreg(X, y, theta, lambdaa):
    J = (-1/m) * (y.T @ np.log(h(X, theta)) + (1 - y.T) @ np.log(1 - h(X, theta)))
    reg = (lambdaa/(2*m)) * (theta[1:].T @ theta[1:])  # by convention, skip theta_0
    J = J + reg
    return J

# previously we've written the following for the cost J; similar outcome, different python notation:
# J = (-1/m) * np.sum(np.multiply(y, np.log(h(X, theta))) + np.multiply((1-y), np.log(1 - h(X, theta))))

The gradient for logistic regression (previous hands-on) has been extended by a regularization term to avoid overfitting. By convention $\Theta_0$ is not used in the regularization term:


<img src="https://github.com/sjoerdteunisse/Avans-A.I-Minor/blob/master/Workshop%202/images/gradientRegularized.png?raw=true"/>


**Exercise**: implement the body of the regularized gradient.

In [ ]:
# gradient
def gradReg(X, y, theta, lambdaa):
    grad = np.zeros([m,1])
    grad = (1/m) * X.T @ (h(X, theta) - y)  # simultaneous update
    grad[1:] = grad[1:] + (lambdaa / m) * theta[1:]
    return grad

In [ ]:
# freehand test of cost function and gradient
theta = np.zeros((n, 1))
lambdaa = 0  # lambda is a python keyword, therefore an additional a in the name
print(Jreg(X, y, theta, lambdaa))  # should be 0.6931471805599453
np.testing.assert_array_almost_equal_nulp(Jreg(X, y, theta, lambdaa), 0.6931471805599453)
print(gradReg(X, y, theta, lambdaa))  # should give [[-0.1] [-12.009216589291151] [-11.262842205513593]]
np.testing.assert_array_almost_equal_nulp(gradReg(X, y, theta, lambdaa), 
      [[0.008474576271186442],
       [0.018788093220338975],
       [7.777118644070071e-05],
       [0.05034463953635592],
       [0.011501330787338978],
       [0.037664847359550876],
       [0.018355987221154245],
       [0.00732393391122216],
       [0.008192444683890361],
       [0.023476488865153248],
       [0.0393486234391602],
       [0.002239239066396939],
       [0.0128600503371337],
       [0.0030959372024053984],
       [0.039302817110394433],
       [0.019970746726922377],
       [0.004329832324171364],
       [0.0033864390190701966],
       [0.0058382207780586105],
       [0.004476290665122478],
       [0.031007984901327713],
       [0.031031244228507674],
       [0.0010974023848666578],
       [0.006315707966420357],
       [0.00040850300602094134],
       [0.007265043164341688],
       [0.0013764617476890438],
       [0.03879363634483877]])

In [ ]:
# learning algorithm for logsitic regression (identical to the one in the hands-on about linear regression!)
def gradientDescent(X, y, theta, lambdaa, alpha, num_iters):
    J_history = []
    for _ in range(num_iters):
        theta -= alpha * gradReg(X, y, theta, lambdaa)
        J_history.append(Jreg(X, y, theta, lambdaa))  # to allow displaying cost as a function of #iters
    return theta, J_history

In [ ]:
# perform learning, but use a more sophisticated optimizer than gradient descent: fmin_tn
# define as function to allow reuse
def learn(X, y, theta, lambdaa):
    import scipy.optimize as opt

    def Jreg_(theta, X, y, lamdaa):
        return Jreg(X, y, theta, lamdaa)
    def gradReg_(theta, X, y, lamdaa):
        return gradReg(X, y, theta, lamdaa)

    output = opt.fmin_tnc(func = Jreg_, x0 = theta.flatten(), fprime = gradReg_, args = (X, y.flatten(), lambdaa))
    theta = output[0][:, np.newaxis]  # transform theta from vector to matrix
    print(theta[0:5]) # only the first 5; should be [ 1.27271026  0.62529965  1.18111686 -2.01987398 -0.91743189]
    return theta

lambdaa = 1
theta = learn(X, y, theta, lambdaa)

In [ ]:
# plot decision boundary (define as function to allow reuse)
def plot_decision_boundary(theta, lambdaa):
    u = np.linspace(-1, 1.5, 50)
    v = np.linspace(-1, 1.5, 50)
    z = np.zeros((len(u), len(v)))

    def mapFeatureForPlotting(X1, X2):
        out = np.ones(1)
        for i in range(1, degree + 1):
            for j in range(i + 1):
                out = np.hstack((out, np.multiply(np.power(X1, i - j), np.power(X2, j))))
        return out

    for i in range(len(u)):
        for j in range(len(v)):
            z[i,j] = np.dot(mapFeatureForPlotting(u[i], v[j]), theta)

    mask = y == 1
    X_ = df.iloc[:,:-1]  # original X
    passed = plt.scatter(X_[mask][0], X_[mask][1])
    failed = plt.scatter(X_[~mask][0], X_[~mask][1])
    plt.contour(u,v,z,0, colors='red')
    plt.xlabel('Microchip Test1')
    plt.ylabel('Microchip Test2')
    plt.legend((passed, failed), ('Passed', 'Failed'))
    plt.title('Decision boundary for lambda=' + str(lambdaa))
    plt.show()

plot_decision_boundary(theta, lambdaa)

In [ ]:
# accuracy (same source code as previous hands-on)
def accuracy(X, y, theta):
    pred = h(X, theta) >= 0.5
    acc = np.mean(np.array(pred) == y)
    return acc * 100

print("accuracy = " + str(accuracy(X, y, theta)))  # should be 83.05%

Let's try different lambdaa's to see the effects of overfitting and underfitting. Plot the decision boundary and calculate the accuracy. Are the results according to your expectation?

**Discussion**: With $\lambda = 0$, we see that indeed the decision boundary is less smooth, which is some indication of overfitting, as the decision boundary is optimized for the training set and will generalize less well for other data. Later in this course, we'll introduce the so-called validation set, which is labeled data that the model is not trained for. When the model does very well for the training set, but not well for the validation set, this is a clear sign of overfitting.

**Exercise**: adding more features, will allow the model to better fit the training set, so there's more risk of overfitting. Try this by setting degree=8 (and put the two tests in comments, as they will fail). This will generate more polynomials, so more features. This will show clearer that model is overfitting. Also note that higher accuracy.

In [ ]:
lambdaas = [0, 1, 100, 150]
for lambdaa in lambdaas:
    theta = np.zeros((n, 1))
    
    # own gradient descent implementation
    num_iters = 150000 # hyper parameter number of iterations
    alpha = 0.002  # hyper parameter learning rate
    #theta, J_history = gradientDescent(X, y, theta, lambdaa, alpha, num_iters)

    # optimized learning algorithm
    theta = learn(X, y, theta, lambdaa)

    plot_decision_boundary(theta, lambdaa)
    print("accuracy = " + str(accuracy(X, y, theta)))